<center>

<p style="font-size:30px;"><strong> Computational Photonics </p>

<p style="font-size:30px;"><strong> Homework 2: Implementation of the Beam Propagation Method </p>


</center>

<center>

**Author:**
*Group 1*
| Name             | Email       |
| -----------      | ----------- |
| *Kiril Armstrong*|  *kiril.armstrong@uni-jena.de*   |
| *Lena Fleischmann*   |  *l.fleischmann@uni-jena.de*           |
| *Yucheng Sun*     |  *yucheng.sun@uni-jena.de*        |

</center>

>**Supervisor:**
>
>*Prof. Thomas Pertsch* 
>
>**Tutor:**
>
>*Tobias Bucher*
>
>*Jan Sperrhake*


In [ ]:
import time
import numpy as np
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt
# from Homework_2_function_headers import waveguide, gauss, beamprop_FN, beamprop_CN, beamprop_BN

# dark bluered colormap, registers automatically with matplotlib on import
import bluered_dark

plt.rcParams.update({
        'figure.figsize': (12/2.54, 9/2.54),
        'figure.subplot.bottom': 0.15,
        'figure.subplot.left': 0.165,
        'figure.subplot.right': 0.925,
        'figure.subplot.top': 0.9,
        'axes.grid': True,
})
plt.close('all')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

**Table of contents**<a id='toc0_'></a>    
- 1. [Introduction](#toc1_)    
- 2. [Beam Propagation Method](#toc2_)    
- 3. [Analysis and Simulation of the Problems](#toc3_)    
  - 3.1. [Task 1 - explicit - implicit scheme](#toc3_1_)    
    - 3.1.1. [Implementation](#toc3_1_1_)    
    - 3.1.2. [Convergence Tests](#toc3_1_2_)    
    - 3.1.3. [Example](#toc3_1_3_)    
  - 3.2. [Task 2 - explicit scheme](#toc3_2_)    
    - 3.2.1. [Implementation](#toc3_2_1_)    
    - 3.2.2. [Convergence Tests](#toc3_2_2_)    
    - 3.2.3. [Example](#toc3_2_3_)    
  - 3.3. [Task 3 - implicit scheme](#toc3_3_)    
    - 3.3.1. [Implementation](#toc3_3_1_)    
    - 3.3.2. [Convergence Tests](#toc3_3_2_)    
    - 3.3.3. [Example](#toc3_3_3_)    
- 4. [Conclusion](#toc4_)    
- 5. [References](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Introduction](#toc0_)

In this project, we employ the beam propagation method to identify the stationary field distribution of a step-index slab waveguide with a Gaussian initial field distribution in 2D (x-z plane). And we use *Python* to implement this method. In addition, both the physical properties and the numerical properties (i.e. the convergence test of the discretization in z direction and also in x direction) of the simulation will be discussed. Specifically, we use three different methods to deal with the problem, i.e. explicit-implicit scheme, explicit scheme and implicit scheme, which corresponds to deal with partial derivative with respect to the z direction using different methods, i.e. central difference, forward difference and backward difference, respectively.

We will only analyze the results got from the explicit-implicit scheme in detail while, for the other two methods, we will only present the general results and some simple analysis.

## 2. <a id='toc2_'></a>[Beam Propagation Method](#toc0_)

The beam propagation method is developed to solve the problem in which the index distribution changes weakly along the propagation direction. 

**1. Approximation**

In this method, the most crucial approximation is the slowly varying envelope approximation. When considering an electromagnetic wave in a homogeneous space, it can be demonstrated that the propagating wave is solely a function of the propagation direction and time, and thus has only transverse components. If weak changes are introduced along the propagation direction, the transverse fields will also undergo slight changes along the two transverse directions, resulting in a longitudinal component due to the divergence condition of the field. Besides, it is also thought that the light is monochromatic and under paraxial approximation. Based on these, we can replace the quickly varying component, $\Phi$, with a slowly varying one $\phi$, as

\begin{align}
\Phi(x, y, z)=\phi(x, y, z)\exp(-ikn_0z), \tag{1}
\end{align}

where $n_0$ is the reference index. 

The starting point is still wave equation derived from Maxwell's equations:

\begin{align}
\nabla^2 \textbf{E} + k^2\epsilon(x, y, z)\textbf{E} = \nabla(\nabla \cdot \textbf{E}). \tag{2} 
\end{align}

**2. Wave equation and divergence condition**

Due to the divergence condition of the electric field, it is enough to only consider the transverse components of the electric field. By separating $\textbf{E}$ and $\nabla$ into transverse components and z component, i.e.

\begin{align}
\textbf{E} &= \textbf{E}_t + E_z\textbf{e}_z, \tag{3} \\
\nabla &= \nabla_t + \textbf{e}_z\frac{\partial}{\partial z}, \tag{4}
\end{align}

we can get the wave equation and divergence condition for transverse components:

\begin{align}
\nabla_{t}^{2}\textbf{E}_{t} + \frac{\partial^2 \textbf{E}_{t}}{\partial z^2} + k^2 \epsilon(x, y, z)\textbf{E}_{t} = \nabla_{t}\Big(\nabla_{t}\cdot \textbf{E}_{t} + \frac{\partial E_z}{\partial z}\Big) \tag{5} 
\end{align}

and the divergence condition: 

\begin{align}
\nabla_t \cdot \Big(\epsilon(x, y, z)\textbf{E}_{t}\Big) + & \frac{\partial \epsilon(x, y, z)}{\partial z} E_z + \epsilon(x, y, z)\frac{\partial E_z}{\partial z} = 0. \tag{6}\\
& \bigg \downarrow slowly\ varying\ along\ the\ z\ direction:\ \frac{\partial \epsilon(x, y, z)}{\partial z} \approx 0 \notag\\
\nabla_t \cdot \Big(\epsilon(x, y, z)\textbf{E}_{t}\Big) + & \epsilon(x, y, z)\frac{\partial E_z}{\partial z} \approx 0. \tag{7}
\end{align}

By further using the divergence condition $(7)$ to eliminate the longitudinal component in the wave equation $(6)$, we can get 

\begin{align}
\nabla_{t}^{2}\textbf{E}_{t} + \frac{\partial^2 \textbf{E}_{t}}{\partial z^2} + k^2 \epsilon(x, y, z)\textbf{E}_{t} = -\nabla_t \Big[\frac{1}{\epsilon(x, y, z)} (\nabla_t\epsilon(x, y, z))\cdot \textbf{E}_{t}\Big]. \tag{8}
\end{align}

Finally, inserting the slowly varying envelope approximation, i.e.

\begin{align}
\textbf{E}_t(x, y, z) = \textbf{e}_t(x, y, z)\exp(-in_0kz), \tag{9}
\end{align}

then the wave equation is given by

\begin{align}
\frac{\partial^2 \textbf{e}_{t}}{\partial z^2} - 2in_0k\frac{\partial \textbf{e}_t}{\partial z} + k^2\Big(\epsilon(x, y, z) - n_0^2\Big)\textbf{e}_t + \nabla_{t}^{2}\textbf{e}_{t} + \nabla_t \Big[\frac{1}{\epsilon(x, y, z)} (\nabla_t\epsilon(x, y, z))\cdot \textbf{e}_{t}\Big] = 0. \tag{10} \\
\bigg \downarrow slowly\ varying\ envelope:\ \Big|\frac{\partial^2 \textbf{e}_{t}}{\partial z^2}\Big| \ll \Big|2in_0k\frac{\partial \textbf{e}_t}{\partial z}\Big| \notag \\
-2in_0k\frac{\partial \textbf{e}_t}{\partial z} + k^2\Big(\epsilon(x, y, z) - n_0^2\Big)\textbf{e}_t + \nabla_{t}^{2}\textbf{e}_{t} + \nabla_t \Big[\frac{1}{\epsilon(x, y, z)} (\nabla_t\epsilon(x, y, z))\cdot \textbf{e}_{t}\Big] = 0. \tag{11}
\end{align}

**3. Scalar approximation**

In this case, we consider translational invariance in the y direction and the incident light is linear polarization along the y direction and the material property is reduced to $\epsilon(x)$. Thus, transverse components is reduced to a scalar field $v(x, z)$. Then, the wave equation is given by:

\begin{align}
-2in_0k\frac{\partial v(x, z)}{\partial z} + k^2\Big(\epsilon(x) &- n_0^2\Big)v(x, z) + \frac{\partial^2 v(x, z)}{\partial x^2} = 0. \tag{12} \\
& \downarrow \notag \\
2in_0k\frac{\partial v(x, z)}{\partial z} = \bigg[k^2\Big(& \epsilon(x) - n_0^2\Big) + \frac{\partial^2}{\partial x^2} \bigg] v(x, z) \tag{13} \\
& \bigg \downarrow \bar{k} \equiv n_0k\ and\ k(x)^2 \equiv k^2\epsilon(x) \notag \\
\frac{\partial v(x, z)}{\partial z} =- \frac{i}{2 \bar{k}} \bigg[\Big(& k(x)^2 - \bar{k}^2\Big) + \frac{\partial^2}{\partial x^2} \bigg] v(x, z) \tag{14}\\
& \bigg \downarrow \mathcal{L}= -\frac{i}{2\bar{k}}\bigg[\Big(k(x)^2 - \bar{k}^2\Big) + \frac{\partial^2}{\partial x^2} \bigg] \tag{15} \\
\frac{\partial v(x, z)}{\partial z} &=\mathcal{L}v(x, z). \tag{16}
\end{align}

**4. Finite difference**

To implement this method numerically, we use finite difference method to deal with the derivatives. We discuss the discretization $\{x_i; \Delta x\}$ of the transverse coordinate and $\{z_n; \Delta z\}$ of the longitudinal coordinate. And there are three options to deal with the partial derivative, i.e. central difference, forward difference, and backward difference.

The function v(x, z) is discretized and given by $v_i^n\equiv v(x_i=i*\Delta x, z_n=n * \Delta z)$. Thus, the wave equation can be represented as

\begin{align}
\frac{\partial v(x, z)}{\partial z}\bigg|_{(x_i, z_n)} &=\mathcal{L}v(x, z)\bigg|_{(x_i, z_n)} \tag{17} \\
&\downarrow \notag\\
\frac{\partial \textbf{v}(x)}{\partial z}\bigg|_{z_n} &=\mathcal{L}\textbf{v}(x)\bigg|_{z_n}, \tag{18}
\end{align}

where $\textbf{v}(x)|_{z_n}$ represents the field distribution vector at $z=z_n$ for all $\{ x_i \}$. In the following, we use $\textbf{v}^n$ to stand for $\textbf{v}(x)|_{z_n}$.

We deal with the derivative with respect to the transverse coordinate using the central difference, which is given by

\begin{align}
\frac{\partial^2 v(x, z)}{\partial x^2}\bigg|_{(x_i, z_n)} \approx \frac{v_{i+1}^{n}-2v_{i}^{n}+v_{i-1}^{n}}{(\Delta x)^2}. \tag{19}
\end{align}

So, the operator $\mathcal{L}$ can be described using the following matrix

\begin{align}
\mathcal{L} = -\frac{j}{2\bar{k}(\Delta x)^2} \left[ \begin{array}{ccccc}
-2 & 1 & 0 & \cdots & 0\\
\\
1 & -2 & 1 & \cdots & 0\\
\\
0 & \ddots & \ddots & \ddots & 0\\
\\
\vdots &  & 1 & -2 & 1\\
\\
0 & \cdots & 0 & 1 & -2 \end{array} \right] + 
j\left[ \begin{array}{ccccc}
W_1 & 0 & \cdots & \cdots & 0\\
\\
0 & \ddots & & & \vdots \\
\\
\vdots & & W_j & & &\\
\\
 & & & \ddots & 0\\
\\
0 & \cdots & & 0 & W_{N_x}\\
\end{array} \right], \tag{20}
\end{align}

where $W_i$ is given by

\begin{align}
W_i = \frac{k^2(x_i)-\bar{k}^2}{2\bar{k}}. \tag{21}
\end{align}

For the partial derivative with respect to the z, we will consider central difference, forward difference, and backward difference, respectively.

From the equation $(18)$, we can have 

- central difference - explicit-implicit scheme

\begin{align}
\frac{\textbf{v}^{n+1} - \textbf{v}^{n}}{ \Delta z} \bigg|_{z_{n+1/2}} = \frac{\textbf{v}^{n+1} - \textbf{v}^{n+1/2} + \textbf{v}^{n+1/2} - \textbf{v}^{n}}{2\cdot(\Delta z/2)} &\bigg|_{z_{n+1/2}} = \frac{1}{2}\bigg[\frac{\textbf{v}^{n+1} - \textbf{v}^{n+1/2}}{\Delta z /2} \bigg|_{z_{n+1}} + \frac{\textbf{v}^{n+1/2} - \textbf{v}^{n}}{\Delta z /2} \bigg|_{z_{n}}\bigg] \notag \\
\notag\\
=\frac{1}{2}\bigg[ \mathcal{L}\textbf{v}^{n+1} &+ \mathcal{L}\textbf{v}^{n}\bigg]\tag{22}\\
&\Big\downarrow \notag \\ 
(\textbf{I} - \Delta z \mathcal{L})\textbf{v}^{n+1} &= (\textbf{I} + \Delta z \mathcal{L})\textbf{v}^{n}, \tag{23}
\end{align}

- forward difference - explicit scheme

\begin{align}
\frac{\textbf{v}^{n+1} - \textbf{v}^{n}}{\Delta z} \bigg|_{z_n} &= \mathcal{L}\textbf{v}^n \tag{24}\\
&\Big\downarrow \notag \\
\textbf{v}^{n+1} &= (\textbf{I} + \Delta z \mathcal{L})\textbf{v}^{n}, \tag{25}
\end{align}

- backward difference - implicit scheme

\begin{align}
\frac{\textbf{v}^{n+1} - \textbf{v}^{n}}{\Delta z} \bigg|_{z_{n+1}} &= \mathcal{L}\textbf{v}^{n+1} \tag{26}\\
&\Big\downarrow \notag \\
(\textbf{I} - \Delta z \mathcal{L})\textbf{v}^{n+1} &= \textbf{v}^{n}. \tag{27}
\end{align}

Our goal is to to solve the equation $(23)$, $(25)$ and $(27)$ numerically. And to solve the field distribution over the whole x-z area, i.e. structure area, we need to iterate to calculate theses equations for each discretized z positions. 

<center>
    
<img src="structure_discretization_and_initial_fields.jpg" alt="structure_discretization_and_initial_fields" width="800" height="400"/>

Figure 2-1: The operation time for different discretized size of z direction.

<center>

## 3. <a id='toc3_'></a>[Analysis and Simulation of the Problems](#toc0_)

- Waveguide profile

In [ ]:
def waveguide(xa, xb, Nx, n_cladding, n_core):
    '''Generates the refractive index distribution of a slab waveguide
    with step profile centered around the origin of the coordinate
    system with a refractive index of n_core in the waveguide region
    and n_cladding in the surrounding cladding area.
    All lengths have to be specified in µm.

    Parameters
    ----------
        xa : float
            Width of calculation window
        xb : float
            Width of waveguide
        Nx : int
            Number of grid points
        n_cladding : float
            Refractive index of cladding
        n_core : float
            Refractive index of core

    Returns
    -------
        n : 1d-array
            Generated refractive index distribution
        x : 1d-array
            Generated coordinate vector
    '''
    
    x = np.linspace(-xa/2, xa/2, Nx)
    n = np.zeros(Nx, dtype=float)
    # index distribution
    for i in range(Nx):
        if abs(x[i]) <= xb/2:
            n[i] = n_core
        else:
            n[i] = n_cladding

    return n, x

The simulated structure is a step-index slab waveguide. This function is used to define the refractive index profile of the structure.  

- Initial field - Gaussian field distribution

In [ ]:
def gauss(xa, Nx, w):
    '''Generates a Gaussian field distribution v = exp(-x^2/w^2) centered
    around the origin of the coordinate system and having a width of w.
    All lengths have to be specified in µm.

    Parameters
    ----------
        xa : float
            Width of calculation window
        Nx : int
            Number of grid points
        w  : float
            Width of Gaussian field

    Returns
    -------
        v : 1d-array
            Generated field distribution
        x : 1d-array
            Generated coordinate vector
    '''
    
    x = np.linspace(-xa/2, xa/2, Nx)
    v = np.exp(-x**2/w**2)
    return v, x

The problem is actually an initial value problem. Thus, we need to set up the initial value, i.e. the field values at the position z = 0. In this problem, we let the initial value to be a gaussian field distribution. This function is used to define the gaussian field distribution at the front end of the waveguide.

- Basic parameters

In [ ]:
# computational parameters
z_end   = 100       # propagation distance
lam     = 1         # wavelength
nd      = 1.455     # reference index
xa      = 50        # size of computational window

# waveguide parameters
xb      = 2.0       # size of waveguide
n_cladding  = 1.45      # cladding index
n_core  = 1.46      # core refr. index

# source width
w       = 5.0       # Gaussian beam width

This part defines the basic parameters of the simulation problem, including the geometry and index values of the slab waveguide and also the important parameters of the initial gaussian field distribution. Please refer to figure 2-1 for a schematic sketch of the assumed parameters.

### 3.1. <a id='toc3_1_'></a>[Task 1 - explicit - implicit scheme](#toc0_)

In this part, we use the explicit-implicit scheme (Crank-Nicolson scheme) to numerically calculate the field distribution of the structure, perform the corresponding convergence tests and obtain the simulation results for the specific parameters, i.e. specific transverse and longitudinal grid.

#### 3.1.1. <a id='toc3_1_1_'></a>[Implementation](#toc0_)

In [ ]:
def beamprop_CN(v_in, lam, dx, n, nd,  z_end, dz, output_step):
    '''Propagates an initial field over a given distance based on the
    solution of the paraxial wave equation in an inhomogeneous
    refractive index distribution using the explicit-implicit
    Crank-Nicolson scheme. All lengths have to be specified in µm.

    Parameters
    ----------
        v_in : 1d-array
            Initial field
        lam : float
            Wavelength
        dx : float
            Transverse step size
        n : 1d-array
            Refractive index distribution
        nd : float
            Reference refractive index
        z_end : float
            Propagation distance
        dz : float
            Step size in propagation direction
        output_step : int
            Number of steps between field outputs

    Returns
    -------
        v_out : 2d-array
            Propagated field
        z : 1d-array
            z-coordinates of field output
    '''
    
    # Basic parameters - wavenumbers
    k0 = 2*np.pi/lam
    kd = nd*k0
    k1 = n*k0
    k2 = np.ones(len(k1)) * kd

    # Construction of the operator matrix L1
    ## Diagonal elements
    diagonals_1 = np.zeros((3, len(n)))
    diagonals_1[0] = np.ones(len(n)) * (-2)
    diagonals_1[1] = np.ones(len(n)) * 1
    diagonals_1[-1] = np.ones(len(n)) * 1
    diag_position_1 = [0, 1, -1]
    ## Sparse matrix construction
    L1 = sps.diags(diagonals_1, diag_position_1)
    L1 = (-1j/(2*kd*dx**2)) * L1

    # Construction of the operator matrix L2
    ## Diagonal elements
    diagonals_2 = np.zeros((1, len(n)))
    diagonals_2[0] = (k1**2 - k2**2) / (2*kd)
    diag_position_2 = [0]
    ## Sparse matrix construction
    L2 = sps.diags(diagonals_2, diag_position_2)
    L2 = -1j*L2

    # Construction of the operator matrix L
    L = L1 + L2

    ## Construction of the operator matrix M1
    M1 = sps.eye(len(n)) - (dz/2) * L
    ## Construction of the operator matrix M2
    M2 = sps.eye(len(n)) + (dz/2) * L

    # Crank-Nicolson scheme
    # Consider output_step with z = linspace(0, z_end, int(z_end/dz) + 1)
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(z), len(n)), dtype=complex)
    # v_out[0,:] = v_in
    # for i in range(len(z) - 1):
    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[i + 1 ,:] = sps.linalg.spsolve(M1, M2.dot(v_out[i,:]))
    #     i += 1


    # Consider output_step with z = z[i] + dz
    z = []
    z.append(0)
    v_out = []
    v_out.append(v_in)
    counter = 1
    i = 0
    for i in range(int(z_end/dz) + 1):
        if z[i] > z_end:
            break
        v_out.append(sps.linalg.spsolve(M1, M2.dot(v_out[counter - 1][:])))
        z.append(z[i] + dz)
        i += 1
        counter += 1
    
    # selection using output_step
    v_out = v_out[::output_step]
    z = z[::output_step]

    return v_out, z

In general, there are two different ways to define the discretization of the z-direction. One is to directly create an equally spaced vector using the code in line 65. This method can change the real discretized distance $dz$, i.e. the distance between two adjacent discretized points is not equal to $dz$. The other is to use the for loop to create an equally spaced vector by adding $dz$ per loop to the previous value. This method can make the discretized distance exactly $dz$ as desired. We have programmed both of these two different methods, the first one (*line 70 ~ 76*) and the second one (*line 80 ~ 92*). The return value v_out in the first method is actually an two dimensional array as requested while that in the second method is a list.

The core here can be divided into three parts:

1. building the operator matrix, *line 40 ~ 66*
   
   As stated in the theoretical part, the influence of the system on the field is operator $\mathcal{L}$ mathematically, which is represented by a tri-diagonal matrix using the finite difference method. To save the computational memories, we use sparse matrix, which is assembled using the *SciPy* function *sps.diags* to create diagonal matrices from the vectors containing the diagonal components and diagonal positions.
    
2. solving the wave equation for each z-position as described in section 2, *line 75 or 89*
   
   We use the *SciPy* function *sps.linalg.spsolve* to solve the wave equation using the initial values. 

3. iteratively solving these equations for each discretized z-position. *line 73 ~ 76 or 86 ~ 92*
   
   To find the field distribution over the whole structure area, we need to iterate the second step to find all the field distribution at each line $(\{x_i\}, z_n)$ using the for loop.

Please figure 2-1 for details.

#### 3.1.2. <a id='toc3_1_2_'></a>[Convergence Tests](#toc0_)

The implemented algorithm depends on two parameters, that determine its accuracy, i.e. the discretized size in the x direction, which is represented by $N_x$ in the problem, and the discretized size in the z direction, which is represented by $dz$ in the problem. Both parameters determine the accuracy of the geometry representation and the accuracy of the finite difference approximation of the differential operator. So we implement the convergence tests for both $N_x$ and $dz$. And these are the same for the explicit and implicit schemes.

**1. Convergence test for dz**

The convergence of the field distribution at the back end in dependence of $dz$ is tested with the following script:

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size
output_step = 1     # output step size


# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# propagation step size - points between 1 and 0.01
dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 0.8, 1.0]

operation_time = np.zeros(len(dz))
field_end = np.zeros((len(dz), Nx))
counter = 0

plt.figure()

for i, dzi in enumerate(dz):
    start = time.time()
    v_out, z = beamprop_CN(v_in, lam, dx, n, nd,  z_end, dzi, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    # Plot results - z direction at x = 0
    plt.plot(z, [np.abs(v[Nx//2])**2 for v in v_out], label=f'dz={dzi:.6f}')
    print("dz = %6.6f, time = %gs" % (dzi, stop - start))
    field_end[counter][:] = np.abs(v_out[-1][:])**2
    counter += 1

plt.xlabel('z [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at x = 0 \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# calculate relative error to the value obtained at highest resolution
real_error = []
for i in range(field_end.shape[0]):
    real_error.append(np.abs(np.linalg.norm(field_end[0][:] - field_end[i][:]) / np.linalg.norm(field_end[0][:])))


# Plot of operation time
plt.figure()
plt.plot(dz, operation_time, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('operation time [s]')
plt.title('Operation time for different dz \n Crank-Nicolson scheme')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(len(dz)):
    plt.plot(x, field_end[i][:], label='dz = %.6f' % dz[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different dz \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(dz, real_error, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('relative error')
plt.title('Relative error for different dz \n Crank-Nicolson scheme')
plt.yscale('log')
plt.show()

In this convergence test, we carefully choose a series of discretized distances $dz$ as shown on *line 15* and output_step to be 1 while keeping the $N_x$ to be a constant as $251$ during the test. The reason we choose these values for $dz$ is to make sure that we can always use the field distribution at the back end to calculate the relative error. A for loop is used to calculate the field distribution of over the whole structure area for each entry of the vector.

<center>
    
<img src="ex-im_dz_time.svg" alt="ex-im_dz_time" width="800" height="400"/>

Figure 3-1: The operation time for different discretized size of z direction.

</center>

With finer discretization along the z direction, the computation time increases exponentially. However, for each $dz$, the exact computation time will change each time the program is run due to the fluctuations of the computational sources. However, the general trend is always the same as shown in figure 3-1.

<center>
    
<img src="ex-im_dz_x.svg" alt="ex-im_dz_x" width="800" height="400"/>

Figure 3-2: Intensity distribution along the z direction at central point of the x direction for different $dz$.

</center>

<center>
    
<img src="ex-im_dz_z-end.svg" alt="ex-im_dz_z-end" width="800" height="400"/>

Figure 3-3: Intensity distribution along the x direction at back end of the waveguide for different $dz$.

</center>

<center>
    
<img src="ex-im_dz_error.svg" alt="ex-im_dz_error" width="800" height="400"/>

Figure 3-4: Relative errors of the intensity distribution at the back end of the waveguide for different $dz$.

</center>

Given that the data set comprises a set of vectors, and as illustrated in  figure 3-2, the most pronounced oscillations occur at the rear of the structure for varying $dz$, it is not possible to evaluate the relative errors of all the vectors directly. Consequently, we have chosen to compare the field distribution at the rear of the structure for each $dz$ and identify a scalar that can be used to characterize each vector. This will then enable us to calculate the relative errors. The value obtained for the finest discretization will be taken as our reference value. To obtain the scalar, we employ *NumPy* function *np.linalg.norm*, which may be effective since it allows for the quantification of the magnitude of differences between vectors in a consistent and meaningful manner, regardless of the dimensions or the individual values in the vectors. Furthermore, to enhance the visibility of the relative errors, we select to utilize logarithmic coordinates for the y axis.

This approach will also be applied to the calculation of the relative errors in the convergence test of Nx.

**2. Convergence test for Nx**

The convergence of the field distribution at the back end in dependence of $N_x$ is tested with the following script:

In [ ]:
# propagation step size
dz = 0.5
output_step = 1


Nx = np.linspace(151, 3151, 61, dtype=int)
operation_time = np.zeros(len(Nx))
field_end = []
x_end = []
for i, Nxi in enumerate(Nx):
    dx      = xa/(Nxi-1) # transverse step size
    start = time.time()
    # create index distribution
    n, x = waveguide(xa, xb, Nxi, n_cladding, n_core)
    # create initial field
    v_in, x     = gauss(xa, Nxi, w)
    v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity
    # propagate field
    v_out, z = beamprop_CN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    x_end.append(x)
    field_end.append(np.abs(v_out[-1][:])**2)
    # print("Nx = %s, time = %gs" % (Nxi, stop - start))

# calculate relative error to the value obtained at highest resolution
real_error = np.zeros(len(Nx))
for i in range(len(Nx)):
    real_error[i] = np.abs(1 - np.linalg.norm(field_end[i])/np.linalg.norm(field_end[-1]))



# Plot of operation time
plt.figure()
plt.plot(Nx, operation_time, 'o-')
plt.xlabel('Nx')
plt.ylabel('operation time [s]')
plt.title('Operation time for different Nx \n Crank-Nicolson scheme')
plt.show()

# Plot results - x direction
plt.figure()
counter = 0
for i in range(len(field_end)):
    if counter > len(field_end):
        break
    plt.plot(x_end[counter], field_end[counter], label='Nx = %d' % Nx[counter])
    counter += 10
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different Nx \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(Nx, real_error, 'o-')
plt.xlabel('Nx')
plt.ylabel('relative error')
plt.title('Relative error for different Nx \n Crank-Nicolson scheme')
plt.show()

In this convergence test, an evenly spaced vector of grid size along the x direction is created, as shown on *line 6*. A for loop is used to calculate the the field distribution over the entire structure. For each entry in the vector we need to re-discretized the index distribution and also the initial gaussian field distribution. 

<center>
    
<img src="ex-im_Nx_time.svg" alt="ex-im_Nx_time" width="800" height="400"/>

Figure 3-5: The operation time for different discretized size of x direction.

</center>

We evaluate the operation time for each $N_x$ including the discretization along the x direction and the calculation time of the field distribution using the function *beamprop_CN*. The result is shown in figure 3-5. The general trend of the operation time is that the operation time increases with finer discretization. However, it fluctuates a lot during a small range of $N_x$.

<center>
    
<img src="ex-im_Nx_z-end.svg" alt="ex-im_Nx_z-end" width="800" height="400"/>

Figure 3-6: Intensity distribution along the x direction at back end of the waveguide for different $N_x$.

</center>

<center>
    
<img src="ex-im_Nx_error.svg" alt="ex-im_Nx_error" width="800" height="400"/>

Figure 3-7: Relative errors of the intensity distribution at the back end of the waveguide for different $N_x$.

</center>

For the error evaluation, we use the same way as the convergence test of $dz$.

#### 3.1.3. <a id='toc3_1_3_'></a>[Example](#toc0_)

After performing the convergence tests on the discretized size along the x and z direction, we now have a look at the field distribution over the whole structure area with a specific discretized grid. We choose the $251\times 201$ discretized grid as a good balance between numerical accuracy and computational costs.

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size

# propagation step size
dz = 0.5
output_step = round(1.0/dz)

# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# calculation
v_out, z = beamprop_CN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
# z = z[::output_step]
for i in range(len(z)):
    v_out[i] = v_out[i]/np.sqrt(np.sum(np.abs(v_out[i])**2)) # normalize power to unity

# Plot results - x-z plane
plt.figure()
plt.pcolormesh(x, z, np.abs(v_out)**2, cmap='bluered_dark')
plt.axvline(x=-xb/2, color='gray', linestyle='--')
plt.axvline(x=xb/2, color='gray', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('z [µm]')
plt.title('Field intensity distribution in the x-z plane of the waveguide \n Crank-Nicolson scheme')
plt.gca().set_aspect('equal')
cb = plt.colorbar()
cb.set_label('intensity')
plt.show()


# Plot results - x direction
plt.figure()
counter = 0
for i in range(0, len(z), 5):
    plt.plot(x, np.abs(v_out[i])**2, label='z = %d' % z[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution in the x direction at different z values \n Crank-Nicolson scheme')
plt.legend()
plt.show()

<center>
    
<img src="ex-im_intensity_area.svg" alt="ex-im_intensity_area" width="800" height="400"/>

Figure 3-8: Intensity distribution over the whole structure area.

</center>

<center>
    
<img src="ex-im_intensity_z.svg" alt="ex-im_intensity_z" width="800" height="400"/>

Figure 3-9: Intensity distribution along the x direction at different discretized z position.

</center>

<center>
    
<img src="ex-im_intensity_x.svg" alt="ex-im_intensity_x" width="800" height="400"/>

Figure 3-10: Intensity distribution along the z direction at central point of the x direction.

</center>

As shown in the figures 3-9 and 3-10, the initial gaussian beam coupled into the core part of the slab waveguide is well bound and propagates through the slab waveguide while that coupled into the cladding part of the waveguide radiates away. And since we do not consider absorption of the material in the project, the central intensity of the waveguide first increases and then decreases during propagation as the spread of the field distribution spreads out. So the energy can be conserved. And to explain this result, let us consider the beam as a composition of different plane waves with wave vectors pointing in different directions. During propagation, only those plane waves corresponding to a small part of different spatial frequency components that satisfy the total internal reflection condition can be bound in the waveguide and propagate through the waveguide. Due to the reduction of the frequency domain, the space domain must be expanded. Generally speaking, these phenomena can all be explained using the diffraction effects by decomposing the field into different frequency components. 

### 3.2. <a id='toc3_2_'></a>[Task 2 - explicit scheme](#toc0_)

In this part, we use the explicit scheme to numerically calculate the field distribution of the structure, perform the corresponding convergence tests and obtain the simulation results for the specific parameters, i.e. specific transverse and longitudinal grid.

For the general description and analysis, please see section 3.1 for details.

#### 3.2.1. <a id='toc3_2_1_'></a>[Implementation](#toc0_)

In [ ]:
def beamprop_FN(v_in, lam, dx, n, nd,  z_end, dz, output_step):
    '''Propagates an initial field over a given distance based on the
    solution of the paraxial wave equation in an inhomogeneous
    refractive index distribution using the explicit scheme. 
    All lengths have to be specified in µm.

    Parameters
    ----------
        v_in : 1d-array
            Initial field
        lam : float
            Wavelength
        dx : float
            Transverse step size
        n : 1d-array
            Refractive index distribution
        nd : float
            Reference refractive index
        z_end : float
            Propagation distance
        dz : float
            Step size in propagation direction
        output_step : int
            Number of steps between field outputs

    Returns
    -------
        v_out : 2d-array
            Propagated field
        z : 1d-array
            z-coordinates of field output
    '''

    # Basic parameters - wavenumbers
    k0 = 2*np.pi/lam
    kd = nd*k0
    k1 = n*k0
    k2 = np.ones(len(k1)) * kd

    # Construction of the operator matrix L1
    ## Diagonal elements
    diagonals_1 = np.zeros((3, len(n)))
    diagonals_1[0] = np.ones(len(n)) * (-2)
    diagonals_1[1] = np.ones(len(n)) * 1
    diagonals_1[-1] = np.ones(len(n)) * 1
    diag_position_1 = [0, 1, -1]
    ## Sparse matrix construction
    L1 = sps.diags(diagonals_1, diag_position_1)
    L1 = (-1j/(2*kd*dx**2)) * L1

    # Construction of the operator matrix L2
    ## Diagonal elements
    diagonals_2 = np.zeros((1, len(n)))
    diagonals_2[0] = (k1**2 - k2**2) / (2*kd)
    diag_position_2 = [0]
    ## Sparse matrix construction
    L2 = sps.diags(diagonals_2, diag_position_2)
    L2 = -1j*L2

    # Construction of the operator matrix L
    L = L1 + L2

    ## Construction of the operator matrix M2
    M2 = sps.eye(len(n)) + (dz/2) * L



    # Explicit scheme
    # Consider output_step with z = linspace(0, z_end, int(z_end/dz) + 1)
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(z), len(n)), dtype=complex)
    # v_out[0,:] = v_in
    # for i in range(len(z) - 1):
    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[i + 1,:] = M2.dot(v_out[i,:])
    #     i += 1

    # Consider output_step with z = z[i] + dz
    z = []
    z.append(0)
    v_out = []
    v_out.append(v_in)
    counter = 1
    i = 0
    
    for i in range(int(z_end/dz) + 1):
        if z[i] > z_end:
            break
        v_out.append(M2.dot(v_out[counter - 1][:]))
        z.append(z[i] + dz)
        i += 1
        counter += 1

    # selection using output_step
    v_out = v_out[::output_step]
    z = z[::output_step]


    return v_out, z

For detail explanation, please see section 3.1.1 .

#### 3.2.2. <a id='toc3_2_2_'></a>[Convergence Tests](#toc0_)

**1. Convergence test for dz**

The convergence of the field distribution at the back end in dependence of $dz$ is tested with the following script:

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size
output_step = 1     # output step size

# propagation step size - 31 points logarithmically spaced between 10 and 0.053
# dz = np.logspace(np.log10(1), np.log10(0.05), 31)
# dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.8, 1.0]
dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 0.8, 1.0]

operation_time = np.zeros(len(dz))
field_end = np.zeros((len(dz), Nx))
counter = 0

plt.figure()

for i, dzi in enumerate(dz):
    start = time.time()
    v_out, z = beamprop_FN(v_in, lam, dx, n, nd,  z_end, dzi, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    # Plot results - z direction at x = 0
    plt.plot(z, [np.abs(v[Nx//2])**2 for v in v_out], label=f'dz={dzi:.6f}')
    print("dz = %6.3f, time = %gs" % (dzi, stop - start))
    field_end[counter][:] = np.abs(v_out[-1][:])**2
    counter += 1

plt.xlabel('z [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at x = 0 \n Explicit scheme')
plt.legend()
plt.show()

# calculate relative error to the value obtained at highest resolution
real_error = []
for i in range(field_end.shape[0]):
    real_error.append(np.linalg.norm(field_end[0][:] - field_end[i][:]) / np.linalg.norm(field_end[0][:]))

# Plot of operation time
plt.figure()
plt.plot(dz, operation_time, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('operation time [s]')
plt.title('Operation time for different dz \n Explicit scheme')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(len(dz)):
    plt.plot(x, field_end[i], label='dz = %.6f' % dz[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different dz \n Explicit scheme')
plt.legend()
plt.show()


# Plot of relative error
plt.figure()
plt.plot(dz, real_error, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('relative error')
plt.title('Relative error for different dz \n Explicit scheme')
# plt.xscale('log')
plt.yscale('log')
plt.show()

<center>
    
<img src="ex_dz_time.svg" alt="ex_dz_time" width="800" height="400"/>

Figure 3-11: The operation time for different discretized size of z direction.

</center>

<center>
    
<img src="ex_dz_x.svg" alt="ex_dz_x" width="800" height="400"/>

Figure 3-12: Intensity distribution along the z direction at central point of the x direction for different $dz$.

</center>

<center>
    
<img src="ex_dz_z-end.svg" alt="ex_dz_z-end" width="800" height="400"/>

Figure 3-13: Intensity distribution along the x direction at back end of the waveguide for different $dz$.

</center>

<center>
    
<img src="ex_dz_error.svg" alt="ex_dz_error" width="800" height="400"/>

Figure 3-14: Relative errors of the intensity distribution at the back end of the waveguide for different $dz$.

</center>

**2. Convergence test for Nx**

The convergence of the field distribution at the back end in dependence of $N_x$ is tested with the following script:

In [ ]:
# # propagation step size
# dz = 0.5
# output_step = 1

# Nx = np.linspace(151, 1151, 101, dtype=int)
# operation_time = np.zeros(len(Nx))
# field_end = []
# x_end = []
# for i, Nxi in enumerate(Nx):
#     dx      = xa/(Nxi-1) # transverse step size
#     start = time.time()
#     # create index distribution
#     n, x = waveguide(xa, xb, Nxi, n_cladding, n_core)
#     # create initial field
#     v_in, x     = gauss(xa, Nxi, w)
#     v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity
#     # propagate field
#     v_out, z = beamprop_FN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
#     stop = time.time()
#     operation_time[i] = stop - start
#     x_end.append(x)
#     field_end.append(np.abs(v_out[-1][:])**2)
#     # print("Nx = %s, time = %gs" % (Nxi, stop - start))

# # calculate relative error to the value obtained at highest resolution
# real_error = np.zeros(len(Nx))
# for i in range(len(Nx)):
#     real_error[i] = np.abs(1 - np.linalg.norm(field_end[i])/np.linalg.norm(field_end[-1]))



# # Plot of operation time
# plt.figure()
# plt.plot(Nx, operation_time, 'o-')
# plt.xlabel('Nx')
# plt.ylabel('operation time [s]')
# plt.title('Operation time for different Nx \n Explicit scheme')
# # plt.xscale('log')
# # plt.yscale('log')
# plt.show()

# # Plot results - x direction
# plt.figure()
# counter = 0
# for i in range(len(field_end)):
#     if counter > len(field_end):
#         break
#     plt.plot(x_end[counter], field_end[counter], label='Nx = %d' % Nx[counter])
#     counter += 5
# plt.axvline(x=-xb/2, color='r', linestyle='--')
# plt.axvline(x=xb/2, color='r', linestyle='--')
# plt.xlabel('x [µm]')
# plt.ylabel('intensity')
# plt.title('Field intensity distribution at far end for different Nx \n Crank-Nicolson scheme')
# plt.legend()
# plt.show()

# # Plot of relative error
# plt.figure()
# plt.plot(Nx, real_error, 'o-')
# plt.xlabel('Nx')
# plt.ylabel('relative error')
# plt.title('Relative error for different Nx \n Explicit scheme')
# # plt.xscale('log')
# # plt.yscale('log')
# plt.show()

We choose not to show the convergence test of $N_x$ in an explicit scheme, since the usual programming method as shown above cannot give a reasonable result. And also for the convergence test of $dz$, the relative error is very large. This may be due to the fact that the explicit scheme is always unstable according to the Von Neumann stability analysis.

#### 3.2.3. <a id='toc3_2_3_'></a>[Example](#toc0_)

After performing the convergence tests on the discretized size along the x and z direction, we now have a look at the field distribution over the whole structure area with a specific discretized grid. We choose the $251\times 201$ discretized grid as a good balance between numerical accuracy and computational costs.

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size

# propagation step size
dz = 0.5
output_step = round(1.0/dz)

# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# calculation
v_out, z = beamprop_FN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
for i in range(len(z)):
    v_out[i] = v_out[i]/np.sqrt(np.sum(np.abs(v_out[i])**2)) # normalize power to unity

# Plot results - x-z plane
plt.figure()
plt.pcolormesh(x, z, np.abs(v_out)**2, cmap='bluered_dark')
plt.axvline(x=-xb/2, color='gray', linestyle='--')
plt.axvline(x=xb/2, color='gray', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('z [µm]')
plt.title('Field intensity distribution in the x-z plane of the waveguide \n Explicit scheme')
plt.gca().set_aspect('equal')
cb = plt.colorbar()
cb.set_label('intensity')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(0, len(z), 5):
    plt.plot(x, np.abs(v_out[i])**2, label='z = %d' % z[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution in the x direction at different z values \n Explicit scheme')
plt.legend()
plt.show()

<center>
    
<img src="ex_intensity_area.svg" alt="ex_intensity_area" width="800" height="400"/>

Figure 3-15: Intensity distribution over the whole structure area.

</center>

<center>
    
<img src="ex_intensity_z.svg" alt="ex_intensity_z" width="800" height="400"/>

Figure 3-16: Intensity distribution along the x direction at different discretized z position.

</center>

<center>
    
<img src="ex_intensity_x.svg" alt="ex_intensity_x" width="800" height="400"/>

Figure 3-17: Intensity distribution along the z direction at central point of the x direction.

</center>

### 3.3. <a id='toc3_3_'></a>[Task 3 - implicit scheme](#toc0_)

In this part, we use the implicit scheme to numerically calculate the field distribution of the structure, perform the corresponding convergence tests and obtain the simulation results for the specific parameters, i.e. specific transverse and longitudinal grid.

For the general description and analysis, please see section 3.1 for details.

#### 3.3.1. <a id='toc3_3_1_'></a>[Implementation](#toc0_)

In [ ]:
def beamprop_BN(v_in, lam, dx, n, nd,  z_end, dz, output_step):
    '''Propagates an initial field over a given distance based on the
    solution of the paraxial wave equation in an inhomogeneous
    refractive index distribution using the implicit scheme. All lengths have to be specified in µm.

    Parameters
    ----------
        v_in : 1d-array
            Initial field
        lam : float
            Wavelength
        dx : float
            Transverse step size
        n : 1d-array
            Refractive index distribution
        nd : float
            Reference refractive index
        z_end : float
            Propagation distance
        dz : float
            Step size in propagation direction
        output_step : int
            Number of steps between field outputs

    Returns
    -------
        v_out : 2d-array
            Propagated field
        z : 1d-array
            z-coordinates of field output
    '''
    
    # Basic parameters - wavenumbers
    k0 = 2*np.pi/lam
    kd = nd*k0
    k1 = n*k0
    k2 = np.ones(len(k1)) * kd

    # Construction of the operator matrix L1
    ## Diagonal elements
    diagonals_1 = np.zeros((3, len(n)))
    diagonals_1[0] = np.ones(len(n)) * (-2)
    diagonals_1[1] = np.ones(len(n)) * 1
    diagonals_1[-1] = np.ones(len(n)) * 1
    diag_position_1 = [0, 1, -1]
    ## Sparse matrix construction
    L1 = sps.diags(diagonals_1, diag_position_1)
    L1 = (-1j/(2*kd*dx**2)) * L1

    # Construction of the operator matrix L2
    ## Diagonal elements
    diagonals_2 = np.zeros((1, len(n)))
    diagonals_2[0] = (k1**2 - k2**2) / (2*kd)
    diag_position_2 = [0]
    ## Sparse matrix construction
    L2 = sps.diags(diagonals_2, diag_position_2)
    L2 = -1j*L2

    # Construction of the operator matrix L
    L = L1 + L2

    ## Construction of the operator matrix M
    M1 = sps.eye(len(n)) - (dz) * L




    # Implicit scheme
    # Consider output_step with z = linspace(0, z_end, int(z_end/dz) + 1)
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(z), len(n)), dtype=complex)
    # v_out[0,:] = v_in
    # for i in range(len(z) - 1):
    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[i + 1,:] = sps.linalg.spsolve(M1, v_out[i,:])
    #     i += 1


    # Consider output_step with z = z[i] + dz
    z = []
    z.append(0)
    v_out = []
    v_out.append(v_in)
    counter = 1
    i = 0
    for i in range(int(z_end/dz) + 1):
        if z[i] > z_end:
            break
        v_out.append(sps.linalg.spsolve(M1, v_out[counter - 1][:]))
        z.append(z[i] + dz)
        i += 1
        counter += 1

    # selection using output_step
    v_out = v_out[::output_step]
    z = z[::output_step]

    return v_out, z


For detail explanation, please see section 3.1.1 .

#### 3.3.2. <a id='toc3_3_2_'></a>[Convergence Tests](#toc0_)

**1. Convergence test for dz**

The convergence of the field distribution at the back end in dependence of $dz$ is tested with the following script:

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size
output_step = 1     # output step size

# propagation step size - 31 points logarithmically spaced between 10 and 0.053
# dz = np.logspace(np.log10(1), np.log10(0.05), 31)
# dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.8, 1.0]
# dz = [0.01, 0.0125, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.078125, 0.1, 0.125, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 1.0]
dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 0.8, 1.0]

operation_time = np.zeros(len(dz))
field_end = np.zeros((len(dz), Nx))
counter = 0

plt.figure()

for i, dzi in enumerate(dz):
    start = time.time()
    v_out, z = beamprop_BN(v_in, lam, dx, n, nd,  z_end, dzi, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    # Plot results - z direction at x = 0
    plt.plot(z, [np.abs(v[Nx//2])**2 for v in v_out], label=f'dz={dzi:.6f}')
    print("dz = %6.3f, time = %gs" % (dzi, stop - start))
    field_end[counter][:] = np.abs(v_out[-1][:])**2
    counter += 1

plt.xlabel('z [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at x = 0 \n Implicit scheme')
plt.legend()
plt.show()

# calculate relative error to the value obtained at highest resolution
real_error = []
for i in range(field_end.shape[0]):
    real_error.append(np.linalg.norm(field_end[0][:] - field_end[i][:]) / np.linalg.norm(field_end[0][:]))


# Plot of operation time
plt.figure()
plt.plot(dz, operation_time, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('operation time [s]')
plt.title('Operation time for different dz \n Implicit scheme')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(len(dz)):
    plt.plot(x, field_end[i], label='dz = %.6f' % dz[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different dz \n Implicit scheme')
plt.legend()
plt.show()


# Plot of relative error
plt.figure()
plt.plot(dz, real_error, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('relative error')
plt.title('Relative error for different dz \n Implicit scheme')
# plt.xscale('log')
plt.yscale('log')
plt.show()


<center>
    
<img src="im_dz_time.svg" alt="im_dz_time" width="800" height="400"/>

Figure 3-18: The operation time for different discretized size of z direction.

</center>

<center>
    
<img src="im_dz_x.svg" alt="im_dz_x" width="800" height="400"/>

Figure 3-19: Intensity distribution along the z direction at central point of the x direction for different $dz$.

</center>

<center>
    
<img src="im_dz_z-end.svg" alt="im_dz_z-end" width="800" height="400"/>

Figure 3-20: Intensity distribution along the x direction at back end of the waveguide for different $dz$.

</center>

<center>
    
<img src="im_dz_error.svg" alt="im_dz_error" width="800" height="400"/>

Figure 3-21: Relative errors of the intensity distribution at the back end of the waveguide for different $dz$.

</center>

**2. Convergence test for Nx**

The convergence of the field distribution at the back end in dependence of $N_x$ is tested with the following script:

In [ ]:
# propagation step size
dz = 0.5
output_step = 1

Nx = np.linspace(151, 3151, 61, dtype=int)
operation_time = np.zeros(len(Nx))
field_end = []
x_end = []
for i, Nxi in enumerate(Nx):
    dx      = xa/(Nxi-1) # transverse step size
    start = time.time()
    # create index distribution
    n, x = waveguide(xa, xb, Nxi, n_cladding, n_core)
    # create initial field
    v_in, x     = gauss(xa, Nxi, w)
    v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity
    # propagate field
    v_out, z = beamprop_BN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    x_end.append(x)
    field_end.append(np.abs(v_out[-1][:])**2)
    # field_end.append(np.abs(v_out[-1]/np.sqrt(np.sum(np.abs(v_out[-1])**2)))**2)
    # print("Nx = %s, time = %gs" % (Nxi, stop - start))

# calculate relative error to the value obtained at highest resolution
real_error = np.zeros(len(Nx))
for i in range(len(Nx)):
    real_error[i] = np.abs(1 - np.linalg.norm(field_end[i])/np.linalg.norm(field_end[-1]))


# Plot of operation time
plt.figure()
plt.plot(Nx, operation_time, 'o-')
plt.xlabel('Nx')
plt.ylabel('operation time [s]')
plt.title('Operation time for different Nx \n Implicit scheme')
plt.show()

# Plot results - x direction
plt.figure()
counter = 0
for i in range(len(field_end)):
    if counter > len(field_end):
        break
    plt.plot(x_end[counter], field_end[counter], label='Nx = %d' % Nx[counter])
    counter += 10
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different Nx \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(Nx, real_error, 'o-')
plt.xlabel('Nx')
plt.ylabel('relative error')
plt.title('Relative error for different Nx \n Implicit scheme')
plt.show()

<center>
    
<img src="im_Nx_time.svg" alt="im_Nx_time" width="800" height="400"/>

Figure 3-22: The operation time for different discretized size of x direction.

</center>

<center>
    
<img src="im_Nx_z-end.svg" alt="im_Nx_z-end" width="800" height="400"/>

Figure 3-23: Intensity distribution along the x direction at back end of the waveguide for different $N_x$.

</center>

<center>
    
<img src="im_Nx_error.svg" alt="im_Nx_error" width="800" height="400"/>

Figure 3-24: Relative errors of the intensity distribution at the back end of the waveguide for different $N_x$.

</center>

#### 3.3.3. <a id='toc3_3_3_'></a>[Example](#toc0_)

After performing the convergence tests on the discretized size along the x and z direction, we now have a look at the field distribution over the whole structure area with a specific discretized grid. We choose the $251\times 201$ discretized grid as a good balance between numerical accuracy and computational costs.

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size

# propagation step size
dz = 0.5
output_step = round(1.0/dz)

# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# calculation
v_out, z = beamprop_BN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
for i in range(len(z)):
    v_out[i] = v_out[i]/np.sqrt(np.sum(np.abs(v_out[i])**2)) # normalize power to unity

# Plot results - x-z plane
plt.figure()
plt.pcolormesh(x, z, np.abs(v_out)**2, cmap='bluered_dark')
plt.axvline(x=-xb/2, color='gray', linestyle='--')
plt.axvline(x=xb/2, color='gray', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('z [µm]')
plt.title('Field intensity distribution in the x-z plane of the waveguide \n Implicit scheme')
plt.gca().set_aspect('equal')
cb = plt.colorbar()
cb.set_label('intensity')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(0, len(z), 5):
    plt.plot(x, np.abs(v_out[i])**2, label='z = %d' % z[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution in the x direction at different z values \n Implicit scheme')
plt.legend()
plt.show()

<center>
    
<img src="im_intensity_area.svg" alt="im_intensity_area" width="800" height="400"/>

Figure 3-25: Intensity distribution over the whole structure area.

</center>

<center>
    
<img src="im_intensity_z.svg" alt="im_intensity_z" width="800" height="400"/>

Figure 3-26: Intensity distribution along the x direction at different discretized z position.

</center>

<center>
    
<img src="im_intensity_x.svg" alt="im_intensity_x" width="800" height="400"/>

Figure 3-27: Intensity distribution along the z direction at central point of the x direction.

</center>

## 4. <a id='toc4_'></a>[Conclusion](#toc0_)

In this project, we identify the stationary field distribution of a step-index slab waveguide, which has a slowly varying index along the z direction, with a Gaussian field distribution at the front end in 2D (x-z plane). We use three different methods to deal with the problem, i.e. explicit-implicit scheme, explicit scheme and implicit scheme.

As seen in the given calculations only the explicit-implicit scheme is valid to compute the field distributions. Especially when having a look at the computed results of the explicit scheme, no trivial and stable field distributions could be achieved. Since the explicit scheme always result in an unstable solution a result like this is expected. Next to this the implicit method gives results which are quite similar to the Crank-Nicolson scheme, which give rise to an stable computational method. Usually this backward difference scheme is not energy conserving this method should not be used for further calculations and discussions.

The explicit-implicit scheme is the only method which is stable and energy conserving. Therefore further discussions will focus on this method.

Firstly, we take a look at the convergence test. For an increase of the step size $dz$ the computational time decrease exponentially, since the number of computed points in the discretization grid will be decreased which will result in a less precise solution. Also the computational error increase for larger discretization length $dz$, because details can not be recorded when the step size is too large. A similar behavior is seen for the convergence test of $N_x$, which implies a change of the discretization length $dx$. Therefore an increase of $N_x$, decrease of $dx$, results in an exponential decrease of the computational error. The computational time increases linearly with an increase of $N_x$.

To optimize the numerical solution and the computational time one should find a sweet spot for the discretized values $dz$ and $N_x$ for a numerical solution that is as accurate and resource-efficient as possible.

The calculated field distribution shows that for the propagation through the waveguide the maximum value slightly decrease. The reason for this can be explained by having a look to the field distributions in the x-y-plane. For larger propagation distances along the z-direction the waves propagate deeper but with less strength into the cladding when compare them to the fields at $z=0$, which therefore result in a decrease of the maximum possible value in the middle of the waveguide.

## 5. <a id='toc5_'></a>[References](#toc0_)

[1]. Thomas Pertsch (2024): Chapter 5 - Beam Propagation Method (BPM). 
   In Thomas Pertsch: Computational Photonics: Abbe School of Photonics, FSU Jena, pp. 59-73.